In [1]:
%load_ext autoreload
%autoreload
import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>
    div.output_area{
        max-height:10000px;
        overflow:scroll;
    }
</style>"""))

import uszipcode
import shapefile
from os.path import join
import numpy as np
import pygeohash as geohash
from timeUtils import clock, elapsed
from shapely.geometry.polygon import Polygon
from shapely.geometry import Point
from random import uniform
basedir="/Users/tgadf/Downloads"

Notebook Last Run Initiated: 2018-08-10 09:45:08.891602


In [4]:
geoID = 4
latID = 13
lngID = 14
wtrID = 11
lndID = 12
ids   = [geoID, latID, lngID, lndID, wtrID]

def splitGeoID(geoID):
    stateID  = geoID[:2]
    shape1ID = geoID[2:5]
    shape2ID = geoID[5:11]
    shape3ID = geoID[11:15]
    return [stateID, shape1ID, shape2ID, shape3ID]

def getRecordInfo(record):
    info = [record[x] for x in ids]
    info[1] = float(info[1])
    info[2] = float(info[2])
    return info

def getBBox(shape):
    bbox = [float('%.6f' % coord) for coord in shape.bbox]
    return bbox

def getShapeGeos(poly, prec, geo, geos, ignores, depth, debug=False):
    if debug:
        print("getShapeGeos({0})".format(geo))
    if geo in ignores:
        return
    
    if depth > 150:
        return
    lat,long  = geohash.decode_exactly(geo)[:2]
    pnt  = Point(long,lat)
    if poly.contains(pnt) is False:
        if debug:
            print("    Adding {0} to list of ignores ({1})".format(geo, len(ignores)))
        ignores.add(geo)
        return
    else:
        geos.add(geo)
        ignores.add(geo)
        if debug:
            print("    Adding {0} to make {1} total geos".format(geo, len(geos)))
        dp = depth+1
        neighbors = set(geohash.neighbors(geo)).difference(ignores)
        #print(depth, len(geos), len(ignores), neighbors)
        for neighbor in list(neighbors):
            if debug:
                print("  Testing {0} from {1} at {2}".format(neighbor, geo, depth))
            getShapeGeos(poly=poly, prec=prec, geo=neighbor, geos=geos, ignores=ignores, depth=dp, debug=debug)
        return

In [5]:
import gzip
import pickle

def save_zipped_pickle(obj, filename, protocol=-1):
    with gzip.open(filename, 'wb') as f:
        pickle.dump(obj, f, protocol)
        
def load_zipped_pickle(filename):
    with gzip.open(filename, 'rb') as f:
        loaded_object = pickle.load(f)
        return loaded_object

In [6]:
def addGeos(geos, geo, init=False):
    nAdd  = 0
    nGeos = len(geos)
    geos.add(geo)
    neighbors = set(geohash.neighbors(geo)).difference(geos)
    for neighbor in list(neighbors):
        geos.add(neighbor)
        nneighbors = set(geohash.neighbors(neighbor)).difference(geos)
        for nneighbor in list(nneighbors):
            geos.add(nneighbor)
    dAdd = len(geos) - nGeos
    if dAdd > 0:
        return 0
    else:
        if init:
            return 0
        return 1
    

def addShapeGeos(shape, prec, geos, debug=True):
    from random import random
    bbox = getBBox(shape)
    lng0      = min([bbox[0], bbox[2]])
    lngrange  = abs(bbox[0] - bbox[2])
    lat0      = min([bbox[1], bbox[3]])
    latrange  = abs(bbox[1] - bbox[3])
    dEW   = 111*latrange
    dNS   = 111*lngrange
    area  = dEW * dNS
    if prec == 5:
        geoarea = 4.9*4.9
    elif prec == 6:
        geoarea = 1.2*0.61
    else:
        geoarea = None
    ngeos = area / geoarea
    if debug:
        print("addShapeGeos(dLat = {0}, dLong = {1}, area = {2}, ngeos = {3})".format(dEW, dNS, area, ngeos))
    nmiss=0
    for i in range(100000):
        genlat = latrange*random() + lat0
        genlng = lngrange*random() + lng0
        geo    = geohash.encode(latitude=genlat, longitude=genlng, precision=prec)
        retval = addGeos(geos, geo)
        if retval == 0:
            nmiss = 0
        else:
            nmiss += 1
        if debug:
            print("\t",i,"\tMiss:\t",nmiss,"Ngeos:\t",len(geos))
        if nmiss > ngeos/2:
            break
    if debug:
        print("  Added {0} geos with {1} misses in the end.".format(nadd, nmiss))
    return 

In [4]:
for stdir in stdirs:
    print("---> {0}".format(stdir))
    fname = '{0}-{1}-geos.p'.format(stdir, prec)
    print(exists(fname))
    data = {}
    statedata = {}
    try:
        stdata = pickle.load(open(fname, 'rb'))
    except:
        print("Could not open {0}".format(fname))
        continue
        
    mtype=stdir.split("_")[0]

    for geoid, geos in stdata.items():
        for geo in geos:
            if geomap.get(geo) is None:
                geomap[geo] = {}
            geomap[geo][mtype] = geoid
            if mtype == "County":
                geomap[geo]["State"] = geoid[:2]


fname="geomap-{0}.p".format(prec)
print("Writing {0}".format(fname))
pickle.dump(geomap,  open(fname, "wb"))

---> Tract_2010Census_DP1
True
---> ZCTA_2010Census_DP1
True
---> County_2010Census_DP1
True
---> CBSA_2010Census_DP1
True
---> CSA_2010Census_DP1
True
---> Place_2010Census_DP1
True
---> MetDiv_2010Census_DP1
True
---> CouSub_2010Census_DP1
True
Writing geomap-5.p


In [26]:
from random import choice
prec=5
print("Running...")
try:
    import pickle
    geomap = pickle.load(open("geomap-{0}.p".format(prec),  "rb"))
except:
    import pickle
    geomap = pickle.load(open("geomap-{0}.p".format(prec),  "rb"))
from uszipcode import ZipcodeSearchEngine
search = ZipcodeSearchEngine()
from haversine import haversine

try:
    statemap = pickle.load(open("statemap.p",  "rb"))
except:
    raise ValueError("No statemap data")
try:
    zzmap = pickle.load(open("zipzctamap.p",  "rb"))
    1/0
except:
    zzmap = {}
try:
    zipmap = pickle.load(open("zipmap.p",  "rb"))
    1/0
except:
    zipmap = {}
try:
    ruralmap = pickle.load(open("ruralmap.p",  "rb"))
    1/0
except:
    ruralmap = {}

notract=set()
nostate=set()
igeo = 0
nzips = 0
start,cmt = clock("Looking over {0} geomap events".format(len(geomap)))
for geo,geodata in geomap.items():
    igeo += 1
    if igeo % 50000 == 0 or igeo == 100 or igeo == 1000 or igeo == 10000 or igeo == 100000:
        elapsed(start, "({0}  {1}/{2}".format(nzips, igeo, len(geomap)), showTime=False)
        pickle.dump(zzmap,  open("zipzctamap.p", "wb"))
        pickle.dump(zipmap,  open("zipmap.p", "wb"))
        pickle.dump(ruralmap,  open("ruralmap.p", "wb"))        
        pickle.dump(notract,  open("notract.p", "wb"))
        pickle.dump(nostate,  open("nostate.p", "wb"))
        print("\tThere are {0} zip codes.".format(len(zipmap)))
        print("\tThere are {0} zip <-> zcta maps.".format(len(zzmap)))
        print("\tThere are {0} rural values.".format(len(ruralmap)))
        
    state = geodata.get("State")
    if state is not None:
        state = choice(list(state))
    tract = geodata.get("Tract")
    if tract is not None:
        tract = choice(list(tract))
    cosub = geodata.get("CouSub")
    if cosub is not None:
        cosub = choice(list(cosub))
    cnty  = geodata.get("County")
    if cnty is not None:
        cnty = choice(list(cnty))
    place = geodata.get("Place")
    if place is not None:
        place = choice(list(place))
    metro = geodata.get("MetDiv")
    if metro is not None:
        metro = choice(list(metro))
    zcta  = geodata.get("ZCTA")
    if zcta is not None:
        zcta = choice(list(zcta))
    if False:
        cd111 = geodata.get("CD111")
        sldl  = geodata.get("SLDL")
        sldu  = geodata.get("SLDU")
        scsd  = geodata.get("SCSD")
        elsd  = geodata.get("ELSD")
        cbsa  = geodata.get("CBSA")
        csa   = geodata.get("CSA")
        unsd  = geodata.get("UNSD")
    city  = None
    rural = None
    zcode = None
    
    if zzmap.get(zcta) is not None and ruralmap.get(zcta) is not None:
        nzips += 1
    else:
        if zcta is not None:
            try:
                zrval = dict(search.by_zipcode(zcta))
                if zrval.get('City') is not None:
                    geomap[geo]['Zip'] = zcode
                    city  = zrval["City"]
                    zcode = zcta
                    zzmap[zcta] = zcode
                    rural = 0.0
                    if state is None:
                        state = statemap[zrval["State"]]
                    if zipmap.get(zcode) is None:
                        zipmap[zcode] = {"Name": zcode, "Area": item["LandArea"]+item["WaterArea"], "Type": item["ZipcodeType"], "Other": city,
                                         "Population": [item["Population"], None, None, None], "Housing": [item["HouseOfUnits"], None], "Occupancy": [None, None]}
            except:
                zcode = None

        if zcode is None:
            nzips += 1
            lat,long  = geohash.decode_exactly(geo)[:2]
            for radius in [1,10,100]:
                res = search.by_coordinate(lat, long, radius=radius)
                for item in res:
                    lattest  = item["Latitude"]
                    longtest = item["Longitude"]
                    dist  = haversine((lat,long),(lattest,longtest))
                    city  = item["City"]
                    zcode = item["Zipcode"]
                    geomap[geo]['Zip'] = zcode
                    zzmap[zcta] = zcode
                    if zipmap.get(zcode) is None:
                        zipmap[zcode] = {"Name": zcode, "Area": item["LandArea"]+item["WaterArea"], "Type": item["ZipcodeType"], "Other": city,
                                         "Population": [item["Population"], None, None, None], "Housing": [item["HouseOfUnits"], None], "Occupancy": [None, None]}
                    rural = dist
                    if state is None:
                        state = statemap[item["State"]]
                    break
                break
            if city is None:
                rural = True
                
        ruralmap[zcta] = rural

    if geomap[geo].get('Zip') is None:
        if zzmap.get(zcta) is not None:
            geomap[geo]['Zip'] = zzmap[zcta]
        else:
            geomap[geo]['Zip'] = None

    if cnty is not None and state is None:
        state = cnty[:2]
        geomap[geo]['State'] = state
    if cosub is not None and not all([cnty,state]):
        if cnty is None:
            cnty  = cosub[:5]
            geomap[geo]['County'] = cnty
        if state is None:
            state = cnty[:2]
            geomap[geo]['State'] = state
    if tract is not None and not all([cnty,state,cosub]):
        if cosub is None:
            cosub  = tract[:10]
            geomap[geo]['CouSub'] = cosub
        if cnty is None:
            cnty = cosub[:5]
            geomap[geo]['County'] = cnty
        if state is None:
            state = cnty[:2]
            geomap[geo]['State'] = state
    if tract is None:
        notract.add(geo)
        #print("Tract {0} is NULL".format(geo))
        #print(geodat
            
    if state is None:
        print('---> No State --->',geo,zcta,zcode,city,rural,geodata)
        nostate.add(geo)
            
fname="geomap-fix-{0}.p".format(prec)
print("Writing {0}".format(fname))
pickle.dump(geomap,  open(fname, "wb"))


pickle.dump(zzmap,  open("zipzctamap.p", "wb"))
pickle.dump(zipmap,  open("zipmap.p", "wb"))
pickle.dump(ruralmap,  open("ruralmap.p", "wb"))
pickle.dump(notract,  open("notract.p", "wb"))
pickle.dump(nostate,  open("nostate.p", "wb"))

Running...
Current Time is Thu Aug 09, 2018 14:24:09 for Looking over 1928486 geomap events
Process [Done with (99  100/1928486] took 0.0 seconds.
	There are 0 zip codes.
	There are 0 zip <-> zcta maps.
	There are 6 rural values.
Process [Done with (999  1000/1928486] took 1.0 seconds.
	There are 0 zip codes.
	There are 0 zip <-> zcta maps.
	There are 8 rural values.
Process [Done with (9999  10000/1928486] took 13.0 seconds.
	There are 0 zip codes.
	There are 0 zip <-> zcta maps.
	There are 11 rural values.
Process [Done with (49999  50000/1928486] took 24.0 seconds.
	There are 0 zip codes.
	There are 3 zip <-> zcta maps.
	There are 31 rural values.
Process [Done with (99999  100000/1928486] took 34.0 seconds.
	There are 0 zip codes.
	There are 3 zip <-> zcta maps.
	There are 35 rural values.
Process [Done with (149999  150000/1928486] took 43.0 seconds.
	There are 0 zip codes.
	There are 3 zip <-> zcta maps.
	There are 35 rural values.
Process [Done with (199999  200000/1928486] took

---> No State ---> 9mxdq None None None None {'CBSA': {'46060'}, 'Zip': '64192'}
---> No State ---> 9mxfk None None None None {'CBSA': {'46060'}, 'Zip': '64192'}
---> No State ---> 9mxd6 None None None None {'CBSA': {'46060'}, 'Zip': '64192'}
---> No State ---> 9t86k None None None None {'CBSA': {'46060'}, 'Zip': '64192'}
---> No State ---> 9mxd7 None None None None {'CBSA': {'46060'}, 'Zip': '64192'}
---> No State ---> 9t8d3 None None None None {'CBSA': {'46060'}, 'Zip': '64192'}
---> No State ---> 9t86q None None None None {'CBSA': {'46060'}, 'Zip': '64192'}
---> No State ---> 9mx6k None None None None {'CBSA': {'46060'}, 'Zip': '64192'}
---> No State ---> 9mxdm None None None None {'CBSA': {'46060'}, 'Zip': '64192'}
---> No State ---> 9t86m None None None None {'CBSA': {'46060'}, 'Zip': '64192'}
---> No State ---> 9t867 None None None None {'CBSA': {'46060'}, 'Zip': '64192'}
---> No State ---> 9mxdk None None None None {'CBSA': {'46060'}, 'Zip': '64192'}
---> No State ---> 9mx6s Non

---> No State ---> f0hxz None None None None {'CBSA': {'42300'}, 'Zip': '64192'}
---> No State ---> f0hwx 49726 None None None {'CBSA': {'42300'}, 'ZCTA': {'49726'}, 'Zip': '49726'}
---> No State ---> f0hxx None None None None {'CBSA': {'42300'}, 'Zip': '64192'}
---> No State ---> f0ks4 None None None None {'CBSA': {'42300'}, 'Zip': '64192'}
---> No State ---> f0k8p None None None None {'CBSA': {'42300'}, 'Zip': '64192'}
---> No State ---> f0ksh None None None None {'CBSA': {'42300'}, 'Zip': '64192'}
---> No State ---> f0hxp None None None None {'CBSA': {'42300'}, 'Zip': '64192'}
---> No State ---> f0hxr None None None None {'CBSA': {'42300'}, 'Zip': '64192'}
---> No State ---> f0kkj None None None None {'CBSA': {'42300'}, 'Zip': '64192'}
---> No State ---> f08eq None None None None {'CBSA': {'26340'}, 'Zip': '64192'}
---> No State ---> f09e2 None None None None {'CBSA': {'26340'}, 'Zip': '64192'}
---> No State ---> f09g3 None None None None {'CBSA': {'26340'}, 'Zip': '64192'}
---> No 

---> No State ---> 9q8de None None None None {'CBSA': {'41860'}, 'CSA': {'488'}, 'MetDiv': {'4186041884'}, 'Zip': '64192'}
---> No State ---> 9q8s4 None None None None {'CBSA': {'41860'}, 'CSA': {'488'}, 'MetDiv': {'4186041884'}, 'Zip': '64192'}
---> No State ---> drqrh None None None None {'CBSA': {'12700'}, 'Zip': '64192'}
---> No State ---> drqrs None None None None {'CBSA': {'12700'}, 'Zip': '64192'}
---> No State ---> drqru None None None None {'CBSA': {'12700'}, 'Zip': '64192'}
---> No State ---> drqrk None None None None {'CBSA': {'12700'}, 'Zip': '64192'}
---> No State ---> drq77 None None None None {'CBSA': {'12700'}, 'Zip': '64192'}
---> No State ---> drtgr None None None None {'CBSA': {'14460'}, 'CSA': {'148'}, 'Zip': '64192'}
---> No State ---> drtfy None None None None {'CBSA': {'14460'}, 'CSA': {'148'}, 'Zip': '64192'}
---> No State ---> drw50 None None None None {'CBSA': {'14460'}, 'CSA': {'148'}, 'Zip': '64192'}
---> No State ---> drw43 None None None None {'CBSA': {'14

---> No State ---> dr58n None None None None {'CBSA': {'35620'}, 'CSA': {'408'}, 'Zip': '64192'}
---> No State ---> dr5cc None None None None {'CBSA': {'35620'}, 'CSA': {'408'}, 'Zip': '64192'}
---> No State ---> dr5fj None None None None {'CBSA': {'35620'}, 'CSA': {'408'}, 'Zip': '64192'}
---> No State ---> drh00 None None None None {'CBSA': {'35620'}, 'CSA': {'408'}, 'Zip': '64192'}
---> No State ---> dr59z None None None None {'CBSA': {'35620'}, 'CSA': {'408'}, 'Zip': '64192'}
---> No State ---> drhed None None None None {'CBSA': {'35620'}, 'CSA': {'408'}, 'Zip': '64192'}
---> No State ---> drhe4 None None None None {'CBSA': {'35620'}, 'CSA': {'408'}, 'Zip': '64192'}
---> No State ---> dr58s None None None None {'CBSA': {'35620'}, 'CSA': {'408'}, 'Zip': '64192'}
---> No State ---> drhjw None None None None {'CBSA': {'35620'}, 'CSA': {'408'}, 'Zip': '64192'}
---> No State ---> drhqh None None None None {'CBSA': {'35620'}, 'CSA': {'408'}, 'MetDiv': {'3562035004'}, 'Zip': '64192'}
--->

...

---> No State ---> 9mg70 None None None None {'CSA': {'348'}, 'Zip': '64192'}
---> No State ---> 9mg4z None None None None {'CSA': {'348'}, 'Zip': '64192'}
---> No State ---> 9mg4f None None None None {'CSA': {'348'}, 'Zip': '64192'}
---> No State ---> 9mffm None None None None {'CSA': {'348'}, 'Zip': '64192'}
---> No State ---> 9mg48 None None None None {'CSA': {'348'}, 'Zip': '64192'}
---> No State ---> 9mg78 None None None None {'CSA': {'348'}, 'Zip': '64192'}
---> No State ---> 9mg4n None None None None {'CSA': {'348'}, 'Zip': '64192'}
---> No State ---> 9mfgt None None None None {'CSA': {'348'}, 'Zip': '64192'}
---> No State ---> 9mg62 None None None None {'CSA': {'348'}, 'Zip': '64192'}
---> No State ---> 9mg1v None None None None {'CSA': {'348'}, 'Zip': '64192'}
---> No State ---> 9mfgz None None None None {'CSA': {'348'}, 'Zip': '64192'}
---> No State ---> 9mg4y None None None None {'CSA': {'348'}, 'Zip': '64192'}
---> No State ---> 9mg5u None None None None {'CSA': {'348'}, 'Z

---> No State ---> 9ug1j None None None None {'CSA': {'204'}, 'Zip': '64192'}
---> No State ---> 9ug4g None None None None {'CSA': {'204'}, 'Zip': '64192'}
---> No State ---> 9ug1g None None None None {'CSA': {'204'}, 'Zip': '64192'}
---> No State ---> 9ufcr None None None None {'CSA': {'204'}, 'Zip': '64192'}
---> No State ---> 9ug5h None None None None {'CSA': {'204'}, 'Zip': '64192'}
---> No State ---> 9ug58 None None None None {'CSA': {'204'}, 'Zip': '64192'}
---> No State ---> 9uffz None None None None {'CSA': {'204'}, 'Zip': '64192'}
---> No State ---> 9ug1d None None None None {'CSA': {'204'}, 'Zip': '64192'}
---> No State ---> 9ug49 None None None None {'CSA': {'204'}, 'Zip': '64192'}
---> No State ---> 9ug1r None None None None {'CSA': {'204'}, 'Zip': '64192'}
---> No State ---> 9ug04 None None None None {'CSA': {'204'}, 'Zip': '64192'}
---> No State ---> 9ug51 None None None None {'CSA': {'204'}, 'Zip': '64192'}
---> No State ---> 9ug5j None None None None {'CSA': {'204'}, 'Z

---> No State ---> 9vh3u None None None None {'CSA': {'288'}, 'Zip': '64192'}
---> No State ---> 9vh8y None None None None {'CSA': {'288'}, 'Zip': '64192'}
---> No State ---> 9vh2t None None None None {'CSA': {'288'}, 'Zip': '64192'}
---> No State ---> 9vh35 None None None None {'CSA': {'288'}, 'Zip': '64192'}
---> No State ---> 9vh9q None None None None {'CSA': {'288'}, 'Zip': '64192'}
---> No State ---> 9vh96 None None None None {'CSA': {'288'}, 'Zip': '64192'}
---> No State ---> 9vh93 None None None None {'CSA': {'288'}, 'Zip': '64192'}
---> No State ---> 9vh6m None None None None {'CSA': {'288'}, 'Zip': '64192'}
---> No State ---> 9vh6s None None None None {'CSA': {'288'}, 'Zip': '64192'}
---> No State ---> 9vh88 None None None None {'CSA': {'288'}, 'Zip': '64192'}
---> No State ---> 9vh68 None None None None {'CSA': {'288'}, 'Zip': '64192'}
---> No State ---> 9vhdw None None None None {'CSA': {'288'}, 'Zip': '64192'}
---> No State ---> 9vhd4 None None None None {'CSA': {'288'}, 'Z

---> No State ---> c44hn None None None None {'Place': {'0286380'}, 'Zip': '64192'}
---> No State ---> 9ue8v None None None None {'Place': {'4841464'}, 'Zip': '64192'}
---> No State ---> c28zr 98244 None None None {'Place': {'5368330'}, 'ZCTA': {'98244'}, 'Zip': '98244'}
---> No State ---> c2exq None None None None {'Place': {'5316585'}, 'Zip': '64192'}
---> No State ---> c2exr None None None None {'Place': {'5316585'}, 'Zip': '64192'}
---> No State ---> c2exm None None None None {'Place': {'5316585'}, 'Zip': '64192'}
---> No State ---> 9mu1v None None None None {'MetDiv': {'3110031084'}, 'Zip': '64192'}
---> No State ---> 9mu1y None None None None {'MetDiv': {'3110031084'}, 'Zip': '64192'}
---> No State ---> 9mu1u None None None None {'MetDiv': {'3110031084'}, 'Zip': '64192'}
---> No State ---> 9mu1z None None None None {'MetDiv': {'3110031084'}, 'Zip': '64192'}
---> No State ---> dhxpq None None None None {'MetDiv': {'3310048424'}, 'Zip': '64192'}
---> No State ---> dhz0w None None N

Writing geomap-fix-5.p


In [ ]:
dict(search.by_zipcode(600540)).get("City")

In [5]:
gX = 'bcdefghjkmnpqrstuvwxyz0123456789'

In [10]:
states="""Alabama	01	AL
Alaska	02	AK
Arizona	04	AZ
Arkansas	05	AR
California	06	CA
Colorado	08	CO
Connecticut	09	CT
Delaware	10	DE
District of Columbia	11	DC
Florida	12	FL
Georgia	13	GA
Hawaii	15	HI
Idaho	16	ID
Illinois	17	IL
Indiana	18	IN
Iowa	19	IA
Kansas	20	KS
Kentucky	21	KY
Louisiana	22	LA
Maine	23	ME
Maryland	24	MD
Massachusetts	25	MA
Michigan	26	MI
Minnesota	27	MN
Mississippi	28	MS
Missouri	29	MO
Montana	30	MT
Nebraska	31	NE
Nevada	32	NV
New Hampshire	33	NH
New Jersey	34	NJ
New Mexico	35	NM
New York	36	NY
North Carolina	37	NC
North Dakota	38	ND
Ohio	39	OH
Oklahoma	40	OK
Oregon	41	OR
Pennsylvania	42	PA
Rhode Island	44	RI
South Carolina	45	SC
South Dakota	46	SD
Tennessee	47	TN
Texas	48	TX
Utah	49	UT
Vermont	50	VT
Virginia	51	VA
Washington	53	WA
West Virginia	54	WV
Wisconsin	55	WI
Wyoming	56	WY"""

In [12]:
statemap={}
statedata={}
for sdata in states.split("\n"):
    vals = sdata.split("\t")
    statemap[vals[-1]] = vals[1]
    statedata[vals[1]] = vals[0]
    

pickle.dump(statemap,  open("statemap.p",  "wb"))    
pickle.dump(statedata,  open("geoState-{0}.p".format(prec),  "wb"))

In [20]:
statemap

{'AL': '01',
 'AK': '02',
 'AZ': '04',
 'AR': '05',
 'CA': '06',
 'CO': '08',
 'CT': '09',
 'DE': '10',
 'DC': '11',
 'FL': '12',
 'GA': '13',
 'HI': '15',
 'ID': '16',
 'IL': '17',
 'IN': '18',
 'IA': '19',
 'KS': '20',
 'KY': '21',
 'LA': '22',
 'ME': '23',
 'MD': '24',
 'MA': '25',
 'MI': '26',
 'MN': '27',
 'MS': '28',
 'MO': '29',
 'MT': '30',
 'NE': '31',
 'NV': '32',
 'NH': '33',
 'NJ': '34',
 'NM': '35',
 'NY': '36',
 'NC': '37',
 'ND': '38',
 'OH': '39',
 'OK': '40',
 'OR': '41',
 'PA': '42',
 'RI': '44',
 'SC': '45',
 'SD': '46',
 'TN': '47',
 'TX': '48',
 'UT': '49',
 'VT': '50',
 'VA': '51',
 'WA': '53',
 'WV': '54',
 'WI': '55',
 'WY': '56'}

In [8]:
from random import choice

In [10]:
x = set([1,23,5])

In [11]:
x

{1, 5, 23}

In [20]:
choice(list(x))
choice(list(x))
choice(list(x))
choice(list(x))
choice(list(None))
choice(list(x))
choice(list(x))
choice(list(x))

TypeError: 'NoneType' object is not iterable